In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

In [ ]:
clf = GradientBoostingClassifier()

params = {
    "n_estimators": [100, 200, 300, 500, 1000],
    "learning_rate": [0.001, 0.01, 0.05, 0.1],
    "max_depth": [3, 5, 10],
    "min_samples_split": [5, 10, 15],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.5, None],
    "random_state": [42]
}

In [ ]:
save_path = '/code/RQ1/results'
dir_train = '/dataset/train.csv'
dir_test = '/dataset/test.csv'
df_train = pd.read_csv(dir_train)
df_test = pd.read_csv(dir_test)
df = pd.concat([df_train, df_test])
df['categoria_rating'] = df['categoria'].astype(str) + "_" + df['rating'].astype(str)

In [ ]:
X = df.drop(columns=['categoria','text','rating','categoria_rating'])
y = df['rating']

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
splits = list(kfold.split(X, df['categoria_rating'])) # usa x pra saber o número de linhas (amostras) e estratifica por categoria_rating

In [ ]:
random_search = RandomizedSearchCV(
    clf,
    param_distributions = params,
    n_iter = 432, # 20% de 2160
    scoring = 'neg_root_mean_squared_error',
    cv = splits,
    random_state = 42,
    verbose = 3)

In [ ]:
random_search.fit(X, y)

In [ ]:
best_params = random_search.best_params_
print(f'Melhores parâmetros: {best_params}')

In [ ]:
def metrics(y_pred, y_test):
      report = metrics.classification_report(y_test, y_pred, target_names=list(map(str, [1,2,3,4,5])), output_dict=True)
      cm = confusion_matrix(y_test, y_pred, labels=[1,2,3,4,5])
      f1_macro = report['macro avg']['f1-score']

      mae = mean_absolute_error(y_test, y_pred)
      rmse = np.sqrt(mean_squared_error(y_test, y_pred))

      y_true_bin = np.array([1 if y in [4,5] else 0 for y in y_test])
      y_pred_bin = np.array([1 if y in [4,5] else 0 for y in y_pred])
      auc = roc_auc_score(y_true_bin, y_pred_bin)

      return report, round(f1_macro,4), cm, round(mae,4), round(rmse,4), round(auc,4)

In [ ]:
all_y_true = []
all_y_pred = []
all_metrics = []

In [ ]:
for i, (train_idx, test_idx) in enumerate(splits):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = clf(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    print(f"\n***FOLD {i + 1}***")
    all_metrics.append((metrics(y_pred, y_test)))

In [ ]:
for i, (report, f1_macro, cm, mae, rmse, auc) in enumerate(all_metrics):
    print(f"\n***FOLD {i + 1}***")
    print(f"\nClassification Report:\n{report}")
    print(f"\nF1 Macro: {f1_macro}")
    print(f"\nConfusion Matrix:\n{cm}")
    print(f"\nMAE: {mae}")
    print(f"\nRMSE: {rmse}")
    print(f"\nAUC: {auc}")


In [ ]:
results = pd.DataFrame(columns = ['MAE','STD_MAE','RMSE','STD_RMSE','AUC','STD_AUC'])
for i, (report, f1_macro, cm, mae, rmse, auc) in enumerate(all_metrics):
    results.loc[i] = [mae, 0, rmse, 0, auc, 0]

In [ ]:
mean_mae = results['MAE'].mean()
mean_rmse = results['RMSE'].mean()
mean_auc = results['AUC'].mean()
std_mae = results['MAE'].std()
std_rmse = results['RMSE'].std()
std_auc = results['AUC'].std()
mean = {'MAE': mean_mae, 'STD_MAE': std_mae, 'RMSE': mean_rmse, 'STD_RMSE': std_rmse, 'AUC': mean_auc, 'STD_AUC': std_auc}
results = pd.concat([results, pd.DataFrame([mean])], ignore_index=True)
results.to_csv(f'{save_path}/{clf.__class__.__name__}_results')